# **1 Install and Import Dependencies**

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# **2. Instantiate Model**

In [ ]:
#BERT model for sentiment calculation
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# **3. Encode and Calculate Sentiment**

In [19]:
tokens = tokenizer.encode('', return_tensors='pt')

In [20]:
result = model(tokens)

In [21]:
#shows sentiment score in logit form
result.logits

tensor([[-0.5486, -0.4852,  0.2038,  0.3929,  0.2652]],
       grad_fn=<AddmmBackward0>)

In [22]:
#decodes logit back into an integer score from 1-5
int(torch.argmax(result.logits))+1

4

# **4. Collect Reviews**

In [25]:
#code for webscraper
r = requests.get('https://www.yelp.com/biz/the-halal-guys-orlando')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

# **5. Load Reviews into DataFrame and Score**

In [35]:
res = pd.DataFrame(np.array(reviews), columns= ['review'])

In [37]:
#removes comments extracted that start with 'Hi', which are typically replies from host restaurant and are not actually reviews
hostcomment = res['review'].str.startswith('Hi')
df = res[~hostcomment]

In [ ]:
df

In [41]:
#function for feeding review into sentiment analysis
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [44]:
sentiment_score(df['review'].iloc[1])

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df